# Atom-Atom Mapping

Atom-atom mapping can help chemists and biologist to get a better understanding of the reaction mechanisms and provide clues for lead compound optimization for medicinal chemists. In this example, we are going to use two sided permutation Procrustes, which has been implemented `procrustes.permutation_2sided`.

The example we are going to use is shown below, which is the atom-atom mapping between 3,3‐dimethylpent‐1‐en‐4‐yne and but‐1‐en‐3‐yne. For clarity, we denote but‐1‐en‐3‐yne molecule **A** and 3,3‐dimethylpent‐1‐en‐4‐yne molecule **B**. What would be most reasonable mapping? If it is not mapping the double bounds and triple bonds respectively, it is not a good matching based on our chemical knowledge.

![Fig. 1 Two organic compounds for atom-atom mapping](../examples/atom_mapping/before_mapping.png "Fig. 1 Two organic compounds for atom-atom mapping")

In order to figure the mapping relationship, one needs to represent the molecules in a matrix format. We will save the nuclear charge as the diagonal elements and the bond orders for off-diagonal ones, which has been depicted below. In this way, the matrix $A \in \mathbb{R}^{7\times7}$ and the matrix $B \in \mathbb{R}^{4 \times4}$ are built, both of which are symmetric. The atoms are also labeled for later discussion.

![Fig. 2 Graphical representation for two molecules.](../examples/atom_mapping/data.png)

In [ ]:
# only run this cell if you need to install the dependices
!pip install git+https://github.com/theochem/procrustes.git@master

In [1]:
# import required libraries
import numpy as np
from procrustes import permutation_2sided

Now we have two matrices $A$ and $B$ for two molecules,

In [2]:
# Define molecule A, but‐1‐en‐3‐yne
A = np.array([[6, 3, 0, 0],
              [3, 6, 1, 0],
              [0, 1, 6, 2],
              [0, 0, 2, 6]])

In [3]:
# Define molecule B, 3,3‐dimethylpent‐1‐en‐4‐yne
B = np.array([[6, 3, 0, 0, 0, 0, 0],
              [3, 6, 1, 0, 0, 0, 0],
              [0, 1, 6, 1, 0, 1, 1],
              [0, 0, 1, 6, 2, 0, 0],
              [0, 0, 0, 2, 6, 0, 0],
              [0, 0, 1, 0, 0, 6, 0],
              [0, 0, 1, 0, 0, 0, 6]])

Now we have both matrix $A$ and $B$ defined and we can use two-sided permutation procrustes to find the optimal transformation operation.

In [4]:
res = permutation_2sided(A, B, 
                         transform_mode='single',
                         remove_zero_col=False, 
                         remove_zero_row=False,
                         scale=False)

In [5]:
# the permutation matrix
res["t"]

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1.]])

The new matrix of molecule A can be obtained with the permutation operation.

In [6]:
# Compute the transformed molecule A
new_A = np.dot(res["t"].T, np.dot(res["new_a"], res["t"])).astype(int)

In [7]:
new_A

array([[6, 3, 0, 0, 0, 0, 0],
       [3, 6, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 6, 2, 0, 0],
       [0, 0, 0, 2, 6, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]])

In [8]:
res["new_b"]

array([[6, 3, 0, 0, 0, 0, 0],
       [3, 6, 1, 0, 0, 0, 0],
       [0, 1, 6, 1, 0, 1, 1],
       [0, 0, 1, 6, 2, 0, 0],
       [0, 0, 0, 2, 6, 0, 0],
       [0, 0, 1, 0, 0, 6, 0],
       [0, 0, 1, 0, 0, 0, 6]])

The computed result is shown in the figure below, generating ideal matching of the
double and triple carbon-carbon bonds. Because molecule $B$ has more atoms than
molecule $A$, $A$ is padded with zeros in order to perform the two-sided permutation
Procrustes calculation. The new matrix representation of $A$ suggests that atom 3
is empty since the third row and third column of $A$ are zero. That is, a virtual atom
3 was added to molecule $A$ to align with atom 3 in molecule $B$. Similarly, atoms 6
and 7 in molecule $B$ do not have meaningful matches in $A$, and are mapped to two
virtual atoms, atom 6 and 7 in molecule $A$. 

<figure>
   <img src="../examples/atom_mapping/atom_mapping1.png" alt="You can use Markdown to add images to Jupyter Notebook files, such as this image of the Markdown logo. Source: Full Stack Python."></a>
   <figcaption> 
   The left image shows the adjacency matrix of ”newly” created molecule $A$ and atom-atom mapping relationship between molecule $A$ and molecule $B$. The adjacency matrix of molecule $A$ with matrix index shown in magenta and newly padded zeros circled in red. 
The right part displays atom-atom matching of molecule $A$ and $B$. The corresponding matching is denoted by dashed red lines linking the numbered atoms. The solid circles indicate the virtual atoms, which
is created to make the perfect mapping by two-sided permutation Procrustes with one transformation.
   </figcaption>
</figure>

Now the alignment makes sense for our chemical intuition.